In [1]:
import xarray as xr
import numpy as np
from stompy.grid import unstructured_grid
from stompy.spatial import wkb2shp
from stompy import utils
import os
import six
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
import stompy.model.delft.waq_scenario as dwaq
from stompy.model.delft import custom_process

import local_config
six.moves.reload_module(local_config)
six.moves.reload_module(dwaq)
six.moves.reload_module(custom_process)

<module 'stompy.model.delft.custom_process' from '/home/rusty/src/stompy/stompy/model/delft/custom_process.py'>

In [3]:
hydro = dwaq.HydroFiles(hyd_path="../v148_jun_28_2016_dwaq_merge/com-v148_jun_28_2016_dwaq_merge.hyd")
hydro.reference_originals=True # for buggy reasons, this needs to be done right away


In [4]:
g=hydro.grid()
if 0:
    g.write_cells_shp("lsb-cells.shp",[('z_bed',-hydro.element_depth())],overwrite=True)

In [5]:
class LsbTracers(local_config.LocalConfig, custom_process.CustomProcesses, dwaq.WaqModel):
    # 24 is fast, but it was creating a lot of noise.
    # 23 became nan after 5 hours.
    integration_option="\n".join([" 22.62 ; integration option",
                                  "    LOWER-ORDER-AT-BOUND NODISP-AT-BOUND",
                                  "    BALANCES-OLD-STYLE BALANCES-GPP-STYLE",
                                  "    BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT",
                                  "    BAL_NOSUPPRESSSPACE BAL_NOSUPPRESSTIME"])
    
    map_timestep=240000 # FIX! Output was still 30 min
    
    def setup(self):
        self.setup_tracers()
        self.setup_monitoring()
        
    def setup_tracer_continuity(self):
        self.substances['continuity']=dwaq.Substance(initial=1.0)
        # This adds a concentration=1.0 boundary condition on all the boundaries.
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
        self.add_bc(all_bcs,'continuity',1.0)

    def setup_tracer_san_jose(self):
        self.substances['san_jose']=dwaq.Substance(initial=0)
        self.add_bc('SAN_JOSE_flow', 'san_jose',1000.0)

    def setup_tracer_palo_alto(self):
        self.substances['palo_alto']=dwaq.Substance(initial=0)
        self.add_bc('PALO_ALTO_flow', 'palo_alto',1000.0)

    def setup_tracer_san_mateo(self):
        self.substances['san_mateo']=dwaq.Substance(initial=0)
        self.add_bc(['san_mateo'], 'san_mateo', 1000.0)

    def setup_tracer_sunnyvale(self):
        self.substances['sunnyvale']=dwaq.Substance(initial=0)
        self.add_bc('SUNNYVALE_flow', 'sunnyvale',1000.0)

    def setup_tracers(self):
        #self.setup_tracer_continuity()
        
        # Conservative tracers
        self.setup_tracer_san_jose()
        self.setup_tracer_san_mateo()
        self.setup_tracer_sunnyvale()
        self.setup_tracer_palo_alto()
        
        self.setup_pond_source()

        self.parameters['nothreads']=3

        # Decaying tracers
        
    def source_polygons(self):
        return wkb2shp.shp2geom("source_polygons.shp")
    def source_polygon(self,name):
        polys = self.source_polygons()
        idx=np.nonzero(polys['name']==name)[0]
        return polys['geom'][idx[0]]
    
    def setup_pond_source(self):
        poly = self.source_polygon("A7A8")

        # Basic diurnal cycle, growth in the pond during the day,
        # constant mortality everywhere
        #   dP/dt = in_pond(x,y) * diurnal(t) * P  - mort * P
        
        self.substances['pond_chl']=dwaq.Substance(initial=1.0)
        
        # create a spatial parameter for InPond
        self.parameters['in_pond'] = self.polygon_to_parameter(poly,0.5)
        
        # create a diurnal pattern that's about 1.0 on average
        times=np.arange( np.datetime64(hydro.time0),
                         np.datetime64(hydro.time0) + np.timedelta64(hydro.t_secs[-1],'s'),
                         np.timedelta64(1,'h'))
        daylight=utils.ideal_solar_rad(times, Imax=3.0)
        self.parameters['diurnal'] = dwaq.ParameterTemporal(times = daylight.time, values=daylight.sol_rad)
        
        # CART creates, among other options, 
        # d age_conc/dt = partial * conc
        # use conc  
        self.custom_CART(conc="in_pond",age_conc="pond_chl",partial="diurnal")

        # This hasn't been working, 
        #self.custom_Decay("pond_chl",0.2)

        # So instead use regular age tracer setup
        self.substances['pond_agec']=dwaq.Substance(initial=0.0)
        self.custom_CART(conc='pond_chl',age_conc="pond_agec",conc_decay=0.05)
                                    
    def polygon_to_parameter(self,poly,value_inside=1.0,value_outside=0.0):
        g=self.hydro.grid()
        values_2d=np.where(g.select_cells_intersecting(poly),
                           value_inside,value_outside)
        
        values_3d=values_2d[self.hydro.seg_to_2d_element]
        return dwaq.ParameterSpatial(values_3d)
    
    def setup_monitoring(self):
        self.add_transects_from_shp("flux_cross_sections_LtoR.shp",add_station=True,naming='name')        
        
    def run(self,force=False):
        assert self.base_path is not None,"Must specify base_path"
        
        if not force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                self.log.info("Run seems to exist -- will not run again")
                return

        self.cmd_write_hydro()
        self.cmd_write_inp()
        self.cmd_delwaq1()
        self.cmd_delwaq2()
        self.cmd_write_nc()        

# /home/rusty/src/data_lsb_tracer: make sure dwaq runs
# /home/rusty/src/data_lsb_tracer_00: add simple tracers, sections.
# ./data_lsb_tracer_01: longer, but needs nc output written.
# data_lsb_tracer_02: longer, slightly truncated end time, lower decay and use
#    the age-tracer call to get decay instead of custom_Decay. add Palo Alto source.
model = LsbTracers(hydro=hydro,base_path="./data_lsb_tracer_02",overwrite=False)

INFO:LsbTracers: start time updated from hydro: 2016-06-28T14:00:00.000000
INFO:LsbTracers: stop time update from hydro: 2016-09-06T14:00:00.000000
INFO:HydroFiles:Inferring 2D elements, 0 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 50000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 100000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 150000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 200000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 250000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 300000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 350000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 400000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 450000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 500000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D elements, 550000 / 1107830 3-D segments
INFO:HydroFiles:Inferring 2D e

In [6]:
# Short run for testing
# model.stop_time = model.start_time + np.timedelta64(60,'D') # still short...
model.stop_time = model.stop_time - np.timedelta64(1,'D') # avoid end-of-record issue

model.setup()

model.run(force=True)

INFO:join_features:0 open strings, 24 simple polygons
INFO:join_features:Building index
/home/rusty/src/stompy/stompy/spatial/join_features.py:514: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  p.join_id=i
/home/rusty/src/stompy/stompy/spatial/join_features.py:516: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  index=STRtree(simple_polys)
INFO:join_features:done building index
INFO:join_features:Examining largest poly left with area=3242487807.773357, 23 potential interiors
/home/rusty/src/stompy/stompy/model/delft/waq_scenario.py:1492: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  input_nodes=[g.select_nodes_nearest(p)
INFO:LsbTracers:Added 17 monitored transects from flux_cross_sections_LtoR.shp


INFO:LsbTracers:Writing hydro data
INFO:HydroFiles:Using .bnd file, not writing out kludgey boundary-links.csv
INFO:HydroFiles:Writing hyd file
INFO:HydroFiles:Bottom depths will be inferred
INFO:HydroFiles:Segment depth will be inferred
INFO:HydroFiles:Bottom depths will be inferred
INFO:HydroFiles:Segment depth will be inferred
INFO:HydroFiles:Writing srf file


INFO:HydroFiles:Writing hydro parameters


Writing paramater spatiotemporal: supporting_file is ../../v148_jun_28_2016_dwaq_merge/v148_jun_28_2016_dwaq_merge-vertdisper.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_jun_28_2016_dwaq_merge/v148_jun_28_2016_dwaq_merge-tau.seg
Writing paramater spatiotemporal: supporting_file is ../../v148_jun_28_2016_dwaq_merge/v148_jun_28_2016_dwaq_merge-salinity.seg


INFO:InpFile:No dispersion arrays, will skip assignment to substances
INFO:InpFile:No velocity arrays, will skip assignment to substances
INFO:HydroFiles:Done setting boundary info


...

INFO:LsbTracers:89.49% Completed
INFO:LsbTracers:Time remaining: 2.552h (Fri Feb 16 04:50:42 2024) 68.21x realtime
INFO:LsbTracers:89.52% Completed
INFO:LsbTracers:Time remaining: 2.544h (Fri Feb 16 04:50:35 2024) 68.22x realtime
INFO:LsbTracers:89.55% Completed
INFO:LsbTracers:Time remaining: 2.536h (Fri Feb 16 04:50:28 2024) 68.23x realtime
INFO:LsbTracers:89.58% Completed
INFO:LsbTracers:Time remaining: 2.529h (Fri Feb 16 04:50:21 2024) 68.23x realtime
INFO:LsbTracers:89.61% Completed
INFO:LsbTracers:Time remaining: 2.521h (Fri Feb 16 04:50:16 2024) 68.24x realtime
INFO:LsbTracers:89.64% Completed
INFO:LsbTracers:Time remaining: 2.514h (Fri Feb 16 04:50:11 2024) 68.25x realtime
INFO:LsbTracers:89.67% Completed
INFO:LsbTracers:Time remaining: 2.506h (Fri Feb 16 04:50:06 2024) 68.26x realtime
INFO:LsbTracers:89.70% Completed
INFO:LsbTracers:Time remaining: 2.498h (Fri Feb 16 04:50:01 2024) 68.27x realtime
INFO:LsbTracers:89.73% Completed
INFO:LsbTracers:Time remaining: 2.491h (Fri Feb

INFO:LsbTracers:Time remaining: 2.012h (Fri Feb 16 04:43:22 2024) 68.80x realtime
INFO:LsbTracers:91.67% Completed
INFO:LsbTracers:Time remaining: 2.005h (Fri Feb 16 04:43:13 2024) 68.81x realtime
INFO:LsbTracers:91.70% Completed
INFO:LsbTracers:Time remaining: 1.997h (Fri Feb 16 04:43:01 2024) 68.82x realtime
INFO:LsbTracers:91.73% Completed
INFO:LsbTracers:Time remaining: 1.990h (Fri Feb 16 04:42:48 2024) 68.83x realtime
INFO:LsbTracers:91.76% Completed
INFO:LsbTracers:Time remaining: 1.982h (Fri Feb 16 04:42:39 2024) 68.83x realtime
INFO:LsbTracers:91.79% Completed
INFO:LsbTracers:Time remaining: 1.975h (Fri Feb 16 04:42:30 2024) 68.84x realtime
INFO:LsbTracers:91.82% Completed
INFO:LsbTracers:Time remaining: 1.967h (Fri Feb 16 04:42:21 2024) 68.85x realtime
INFO:LsbTracers:91.85% Completed
INFO:LsbTracers:Time remaining: 1.960h (Fri Feb 16 04:42:13 2024) 68.86x realtime
INFO:LsbTracers:91.88% Completed
INFO:LsbTracers:Time remaining: 1.953h (Fri Feb 16 04:42:07 2024) 68.87x realtim

INFO:LsbTracers:93.81% Completed
INFO:LsbTracers:Time remaining: 1.478h (Fri Feb 16 04:37:09 2024) 69.38x realtime
INFO:LsbTracers:93.84% Completed
INFO:LsbTracers:Time remaining: 1.470h (Fri Feb 16 04:37:04 2024) 69.38x realtime
INFO:LsbTracers:93.87% Completed
INFO:LsbTracers:Time remaining: 1.463h (Fri Feb 16 04:36:57 2024) 69.39x realtime
INFO:LsbTracers:93.90% Completed
INFO:LsbTracers:Time remaining: 1.456h (Fri Feb 16 04:36:46 2024) 69.40x realtime
INFO:LsbTracers:93.93% Completed
INFO:LsbTracers:Time remaining: 1.448h (Fri Feb 16 04:36:41 2024) 69.41x realtime
INFO:LsbTracers:93.96% Completed
INFO:LsbTracers:Time remaining: 1.441h (Fri Feb 16 04:36:36 2024) 69.41x realtime
INFO:LsbTracers:93.99% Completed
INFO:LsbTracers:Time remaining: 1.434h (Fri Feb 16 04:36:31 2024) 69.42x realtime
INFO:LsbTracers:94.02% Completed
INFO:LsbTracers:Time remaining: 1.426h (Fri Feb 16 04:36:25 2024) 69.43x realtime
INFO:LsbTracers:94.05% Completed
INFO:LsbTracers:Time remaining: 1.419h (Fri Feb

INFO:LsbTracers:Time remaining: 0.959h (Fri Feb 16 04:31:03 2024) 69.91x realtime
INFO:LsbTracers:95.98% Completed
INFO:LsbTracers:Time remaining: 0.952h (Fri Feb 16 04:31:04 2024) 69.92x realtime
INFO:LsbTracers:96.01% Completed
INFO:LsbTracers:Time remaining: 0.945h (Fri Feb 16 04:31:05 2024) 69.93x realtime
INFO:LsbTracers:96.04% Completed
INFO:LsbTracers:Time remaining: 0.938h (Fri Feb 16 04:31:09 2024) 69.94x realtime
INFO:LsbTracers:96.07% Completed
INFO:LsbTracers:Time remaining: 0.930h (Fri Feb 16 04:31:08 2024) 69.94x realtime
INFO:LsbTracers:96.11% Completed
INFO:LsbTracers:Time remaining: 0.921h (Fri Feb 16 04:30:55 2024) 69.95x realtime
INFO:LsbTracers:96.14% Completed
INFO:LsbTracers:Time remaining: 0.914h (Fri Feb 16 04:30:47 2024) 69.96x realtime
INFO:LsbTracers:96.17% Completed
INFO:LsbTracers:Time remaining: 0.907h (Fri Feb 16 04:30:34 2024) 69.97x realtime
INFO:LsbTracers:96.20% Completed
INFO:LsbTracers:Time remaining: 0.899h (Fri Feb 16 04:30:26 2024) 69.97x realtim

INFO:LsbTracers:98.13% Completed
INFO:LsbTracers:Time remaining: 0.440h (Fri Feb 16 04:25:19 2024) 70.44x realtime
INFO:LsbTracers:98.16% Completed
INFO:LsbTracers:Time remaining: 0.433h (Fri Feb 16 04:25:15 2024) 70.44x realtime
INFO:LsbTracers:98.19% Completed
INFO:LsbTracers:Time remaining: 0.425h (Fri Feb 16 04:25:14 2024) 70.45x realtime
INFO:LsbTracers:98.22% Completed
INFO:LsbTracers:Time remaining: 0.418h (Fri Feb 16 04:25:14 2024) 70.46x realtime
INFO:LsbTracers:98.25% Completed
INFO:LsbTracers:Time remaining: 0.411h (Fri Feb 16 04:25:14 2024) 70.46x realtime
INFO:LsbTracers:98.28% Completed
INFO:LsbTracers:Time remaining: 0.404h (Fri Feb 16 04:25:11 2024) 70.47x realtime
INFO:LsbTracers:98.31% Completed
INFO:LsbTracers:Time remaining: 0.397h (Fri Feb 16 04:25:04 2024) 70.48x realtime
INFO:LsbTracers:98.34% Completed
INFO:LsbTracers:Time remaining: 0.390h (Fri Feb 16 04:24:55 2024) 70.49x realtime
INFO:LsbTracers:98.37% Completed
INFO:LsbTracers:Time remaining: 0.383h (Fri Feb

In [7]:
# Assuming that works, for each of the files that would usually get symlinked...
# a. add override for xxx_filename() in HydroFiles
# b. update HydroFiles.write_xxx() to check self.reference_originals.

# check for any other places that consult enable_write_symlink

# vol, are, flo, flowgeom, 
# what about ParameterSpatioTemporal? I think it's fixed.

# running slowly, either because of other CPU usage, or hitting S:.
# Have to be careful about when 